In [40]:
# Import Packages
import numpy as np
import pandas as pd
import sys
import json
import csv
import networkx as nx
import matplotlib.pyplot as plt

# filepaths

new_node_filepath  = "./Downloads/Translator/ehr_risk_nodes_data_2022_04_05.tsv"
new_edge_filepath = "./Downloads/Translator/ehr_risk_edges_data_2022_04_05.tsv"

### Loading the edge data for checking

In [71]:
tsv_edge_df = pd.read_csv(new_edge_filepath, sep=',')

tsv_edge_df.head(4)

,subject,predicate,object,relation,provided_by,provided_date,category,classifier,auc_roc,feature_importance,feature_coefficient,log_positive_patient_count,log_negative_patient_count
0,CHEBI:2611,biolink:associated_with_increased_likelihood_o...,HP:0000739,RO:0003308,EHR Risk Provider (Multiomics),2021-05-07,biolink:Association,Logistic Regression,0.87615,NaN,3.719874,5,7
1,CHEBI:6539,biolink:associated_with_increased_likelihood_o...,HP:0000739,RO:0003308,EHR Risk Provider (Multiomics),2021-05-07,biolink:Association,Logistic Regression,0.87615,NaN,3.354957,5,7
2,CHEBI:3223,biolink:associated_with_increased_likelihood_o...,HP:0000739,RO:0003308,EHR Risk Provider (Multiomics),2021-05-07,biolink:Association,Logistic Regression,0.87615,NaN,3.023576,5,7
3,MONDO:0002050,biolink:associated_with_increased_likelihood_o...,HP:0000739,RO:0003308,EHR Risk Provider (Multiomics),2021-05-07,biolink:Association,Logistic Regression,0.87615,NaN,2.869545,5,7


In [72]:
tsv_edge_df["predicate"].unique()

array(['biolink:associated_with_increased_likelihood_of_concurrent',
       'biolink:associated_with_decreased_likelihood_of_concurrent'],
      dtype=object)

In [73]:
tsv_edge_df.isna().sum()

subject                          0
predicate                        0
object                           0
relation                         0
provided_by                      0
provided_date                    0
category                         0
classifier                       0
auc_roc                          0
feature_importance            1000
feature_coefficient              0
log_positive_patient_count       0
log_negative_patient_count       0
dtype: int64

In [74]:
tsv_edge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   subject                     1000 non-null   object 
 1   predicate                   1000 non-null   object 
 2   object                      1000 non-null   object 
 3   relation                    1000 non-null   object 
 4   provided_by                 1000 non-null   object 
 5   provided_date               1000 non-null   object 
 6   category                    1000 non-null   object 
 7   classifier                  1000 non-null   object 
 8   auc_roc                     1000 non-null   float64
 9   feature_importance          0 non-null      float64
 10  feature_coefficient         1000 non-null   float64
 11  log_positive_patient_count  1000 non-null   int64  
 12  log_negative_patient_count  1000 non-null   int64  
dtypes: float64(3), int64(2), object(8)

In [75]:
tsv_edge_df.describe()

,auc_roc,feature_importance,feature_coefficient,log_positive_patient_count,log_negative_patient_count
count,1000.000000,0.0,1000.000000,1000.000000,1000.0
mean,0.893773,NaN,0.113537,4.812000,7.0
std,0.036644,NaN,1.363429,0.390908,0.0
min,0.876150,NaN,-16.197925,4.000000,7.0
25%,0.876150,NaN,-0.301279,5.000000,7.0
50%,0.876150,NaN,0.201322,5.000000,7.0
75%,0.876150,NaN,0.635720,5.000000,7.0
max,0.969891,NaN,7.809234,5.000000,7.0


In [76]:
# {"log_positive_patient_count"} to total number of people with subject_id
# "log_negative_patient_coun" to total number of people without subject_id

In [77]:
tsv_edge_df.rename(columns = {"log_negative_patient_count": "log_of_total_negative_people", 
               "log_positive_patient_count": "log_of_total_positive_people"}, inplace = True)

In [78]:
tsv_edge_df.head(4)

,subject,predicate,object,relation,provided_by,provided_date,category,classifier,auc_roc,feature_importance,feature_coefficient,log_of_total_positive_people,log_of_total_negative_people
0,CHEBI:2611,biolink:associated_with_increased_likelihood_o...,HP:0000739,RO:0003308,EHR Risk Provider (Multiomics),2021-05-07,biolink:Association,Logistic Regression,0.87615,NaN,3.719874,5,7
1,CHEBI:6539,biolink:associated_with_increased_likelihood_o...,HP:0000739,RO:0003308,EHR Risk Provider (Multiomics),2021-05-07,biolink:Association,Logistic Regression,0.87615,NaN,3.354957,5,7
2,CHEBI:3223,biolink:associated_with_increased_likelihood_o...,HP:0000739,RO:0003308,EHR Risk Provider (Multiomics),2021-05-07,biolink:Association,Logistic Regression,0.87615,NaN,3.023576,5,7
3,MONDO:0002050,biolink:associated_with_increased_likelihood_o...,HP:0000739,RO:0003308,EHR Risk Provider (Multiomics),2021-05-07,biolink:Association,Logistic Regression,0.87615,NaN,2.869545,5,7


In [79]:
tsv_edge_df[tsv_edge_df["feature_coefficient"]<0].head()

,subject,predicate,object,relation,provided_by,provided_date,category,classifier,auc_roc,feature_importance,feature_coefficient,log_of_total_positive_people,log_of_total_negative_people
398,CHEBI:2679,biolink:associated_with_decreased_likelihood_o...,HP:0000739,RO:0003308,EHR Risk Provider (Multiomics),2021-05-07,biolink:Association,Logistic Regression,0.87615,NaN,-0.002321,5,7
399,HP:0002919,biolink:associated_with_decreased_likelihood_o...,HP:0000739,RO:0003308,EHR Risk Provider (Multiomics),2021-05-07,biolink:Association,Logistic Regression,0.87615,NaN,-0.002416,5,7
400,MONDO:0004866,biolink:associated_with_decreased_likelihood_o...,HP:0000739,RO:0003308,EHR Risk Provider (Multiomics),2021-05-07,biolink:Association,Logistic Regression,0.87615,NaN,-0.004138,5,7
401,MONDO:0007256,biolink:associated_with_decreased_likelihood_o...,HP:0000739,RO:0003308,EHR Risk Provider (Multiomics),2021-05-07,biolink:Association,Logistic Regression,0.87615,NaN,-0.004771,5,7
402,CHEBI:8357,biolink:associated_with_decreased_likelihood_o...,HP:0000739,RO:0003308,EHR Risk Provider (Multiomics),2021-05-07,biolink:Association,Logistic Regression,0.87615,NaN,-0.007995,5,7


In [80]:
tsv_edge_df[(tsv_edge_df["feature_coefficient"] >= 0.0001) &(tsv_edge_df["feature_coefficient"] < 0.123)]

,subject,predicate,object,relation,provided_by,provided_date,category,classifier,auc_roc,feature_importance,feature_coefficient,log_of_total_positive_people,log_of_total_negative_people
342,HP:0001681,biolink:associated_with_increased_likelihood_o...,HP:0000739,RO:0003308,EHR Risk Provider (Multiomics),2021-05-07,biolink:Association,Logistic Regression,0.87615,NaN,0.117818,5,7
343,HP:0020062,biolink:associated_with_increased_likelihood_o...,HP:0000739,RO:0003308,EHR Risk Provider (Multiomics),2021-05-07,biolink:Association,Logistic Regression,0.87615,NaN,0.116147,5,7
344,CHEBI:3309,biolink:associated_with_increased_likelihood_o...,HP:0000739,RO:0003308,EHR Risk Provider (Multiomics),2021-05-07,biolink:Association,Logistic Regression,0.87615,NaN,0.115703,5,7
345,CHEBI:30769,biolink:associated_with_increased_likelihood_o...,HP:0000739,RO:0003308,EHR Risk Provider (Multiomics),2021-05-07,biolink:Association,Logistic Regression,0.87615,NaN,0.115604,5,7
346,HP:0002905,biolink:associated_with_increased_likelihood_o...,HP:0000739,RO:0003308,EHR Risk Provider (Multiomics),2021-05-07,biolink:Association,Logistic Regression,0.87615,NaN,0.114141,5,7
347,CHEBI:8080,biolink:associated_with_increased_likelihood_o...,HP:0000739,RO:0003308,EHR Risk Provider (Multiomics),2021-05-07,biolink:Association,Logistic Regression,0.87615,NaN,0.111740,5,7
348,CHEBI:176843,biolink:associated_with_increased_likelihood_o...,HP:0000739,RO:0003308,EHR Risk Provider (Multiomics),2021-05-07,biolink:Association,Logistic Regression,0.87615,NaN,0.109976,5,7
349,MONDO:0019391,biolink:associated_with_increased_likelihood_o...,HP:0000739,RO:0003308,EHR Risk Provider (Multiomics),2021-05-07,biolink:Association,Logistic Regression,0.87615,NaN,0.108066,5,7
350,MONDO:0024333,biolink:associated_with_increased_likelihood_o...,HP:0000739,RO:0003308,EHR Risk Provider (Multiomics),2021-05-07,biolink:Association,Logistic Regression,0.87615,NaN,0.108028,5,7
351,HP:0003124,biolink:associated_with_increased_likelihood_o...,HP:0000739,RO:0003308,EHR Risk Provider (Multiomics),2021-05-07,biolink:Association,Logistic Regression,0.87615,NaN,0.104295,5,7


## Loading the node data to check the conent

In [81]:
tsv_node_df = pd.read_csv(new_node_filepath, sep = ',')
tsv_node_df.head()

,id,name,category,xref,provided_by
0,HP:0025181,Abdominal aseptic abscess,biolink:PhenotypicFeature,NaN,EHR Risk Provider (Multiomics)
1,HP:0002027,Abdominal pain,biolink:PhenotypicFeature,NaN,EHR Risk Provider (Multiomics)
2,HP:0003115,Abnormal EKG,biolink:PhenotypicFeature,NaN,EHR Risk Provider (Multiomics)
3,HP:0006919,"Abnormal aggressive, impulsive or violent beha...",biolink:PhenotypicFeature,NaN,EHR Risk Provider (Multiomics)
4,HP:0003119,Abnormal circulating lipid concentration,biolink:PhenotypicFeature,NaN,EHR Risk Provider (Multiomics)


In [85]:
# check the unique prefixes
rt = [w.split(":")[0] for w in tsv_node_df["id"].unique()]
set(rt)

{'CHEBI', 'HP', 'MONDO', 'NCIT', 'NONE', 'SNOMEDCT', 'UNII'}

# Parser Function

In [65]:
def load_tsv_data(edge_filepath, node_filepath):
    """
    This function takes two parameters: edge_filepath and the node_filepath.
    It will return a json formats that has the key of id, subject, predicate, object as the main keys. 
    """
    edges_data = pd.read_csv(edge_filepath, sep = ',')
    nodes_data = pd.read_csv(node_filepath, sep = ',')
    # generate the nodes atributes
    node_name_mapping = {}
    node_type_mapping = {}
    node_xref_mapping = {}
    for index, row in nodes_data.iterrows():
        node_name_mapping[row["id"]] = row["name"]
        node_type_mapping[row["id"]] = row["category"].split(':')[1] if str(row["category"]).startswith("biolink:") else row["category"]
        node_xref_mapping[row["id"]] = row["xref"]
        
    # generate the edges attributes
    
    for index, row in edges_data.iterrows():
        if row["subject"] and row["predicate"] and row["subject"].split(':')[0] and row["object"].split(':')[0]:
            # Specify properties for subject
            subject = {
                row["subject"].split(':')[0]: row["subject"],
                "id": row["subject"],
                "name": node_name_mapping[row["subject"]],
                "type": node_type_mapping[row["subject"]]
            }
#             current_xref = node_xref_mapping[row["subject"]]
            if ~np.isnan(node_xref_mapping[row["subject"]]):
                subject[node_xref_mapping[row["subject"]].split(':')[0]] =  node_xref_mapping[row["subject"]]
            
            # Specify properties for object
            objects = {
                row["object"].split(':')[0]: row["object"],
                "id": row["object"],
                "name": node_name_mapping[row["object"]],
                "type": node_type_mapping[row["object"]]
            }
            if ~np.isnan(node_xref_mapping[row["object"]]):
                objects[node_xref_mapping[row["object"]].split(':')[0]] =  node_xref_mapping[row["object"]]
            
            # Specify properties for predicate
            
            predicate = {
                "type": row["predicate"].split(':')[-1] if row["predicate"].startswith("biolink:") else row["predicate"],
                "original": row["predicate"],
                "provided_by": row["provided_by"],
                "provided_date": row["provided_date"],
                "provenance": "https://github.com/NCATSTranslator/Translator-All/wiki/EHR-Risk-KP",
                "category": row["category"].split(':')[1] if row["category"].startswith("biolink:") else row["category"],
                "classifier": row["classifier"],
                "relation": row["relation"],
                "auc_roc": float(row["auc_roc"])
            }
            
            # conditional properties for predicate (if not Null)
            if ~np.isnan(row["feature_importance"]):
                predicate["feature_importance"] =  float(row["feature_importance"])
            if ~np.isnan(row["feature_coefficient"]):
                predicate["feature_coefficient"] = float(row["feature_coefficient"])
                
            # make a unique id from  subject, predicate, and object indentifiers
            unique_id_list = [row["subject"], row["predicate"], row["object"], row["classifier"], str(row["auc_roc"]),
                           str(row["feature_importance"]), str(row["feature_coefficient"])]
        json = {
            "id":'EHR_Risk_KP-'.join(unique_id_list),
            "subject": subject,
            "predicate": predicate,
            "object": objects
        }
        yield json
        

In [66]:
# id: subject ID prefix, subject category, predicate, object ID prefix, object category.
# Yao Yao >> a person to contact for the json file format.

In [67]:
def main():
    count = 0
    verbose = True
    for row in load_tsv_data(new_edge_filepath, new_node_filepath):
        if verbose:
            print(json.dumps(row, sort_keys= False, indent=2))
        count += 1
        if count >= 10:
           break

if __name__ == "__main__": 
    main()      


{
  "id": "CHEBI:2611EHR_Risk_KP-biolink:associated_with_increased_likelihood_of_concurrentEHR_Risk_KP-HP:0000739EHR_Risk_KP-Logistic RegressionEHR_Risk_KP-0.8761500144827434EHR_Risk_KP-nanEHR_Risk_KP-3.719874148814836",
  "subject": {
    "CHEBI": "CHEBI:2611",
    "id": "CHEBI:2611",
    "name": "alprazolam",
    "type": "ChemicalSubstance"
  },
  "predicate": {
    "type": "associated_with_increased_likelihood_of_concurrent",
    "original": "biolink:associated_with_increased_likelihood_of_concurrent",
    "provided_by": "EHR Risk Provider (Multiomics)",
    "provided_date": "2021-05-07",
    "provenance": "https://github.com/NCATSTranslator/Translator-All/wiki/EHR-Risk-KP",
    "category": "Association",
    "classifier": "Logistic Regression",
    "relation": "RO:0003308",
    "auc_roc": 0.8761500144827434,
    "feature_coefficient": 3.719874148814836
  },
  "object": {
    "HP": "HP:0000739",
    "id": "HP:0000739",
    "name": "Anxiety",
    "type": "PhenotypicFeature"
  }
}
{
 

In [86]:
pwd

'/Users/basazinbelhu'